In [36]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data # 获取迭代数据
from torch.utils.data import Dataset,TensorDataset,DataLoader,random_split
from torch.autograd import Variable # 获取变量

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import datetime

In [53]:
#setting
batch_size = 16
learning_rate = 0.01
epochs = 10
clip = 0.01

## load data

In [14]:
all_data = np.load("../input_data/data.npy")
all_label = np.load("../input_data/label.npy")

In [17]:
all_data = torch.from_numpy(all_data)
all_label = torch.from_numpy(all_label)
all_data=all_data.float()
all_label=all_label.long()
dataset=TensorDataset(all_data,all_label)


In [25]:
#划分训练集和测试集
ratio = 0.8
seed = 30
num_train = int(len(dataset) * ratio)
num_test = len(dataset) - num_train
    
train_dataset, validate_dataset = random_split(dataset, [num_train, num_test],torch.Generator().manual_seed(seed))

In [43]:
#Load to DataLoader
print("train_dataset:",len(train_dataset))
print("validate_dataset:",len(validate_dataset))
print("batch_size:",batch_size)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validate_dataset,batch_size=batch_size, shuffle=True)


train_dataset: 1221
validate_dataset: 306
batch_size: 16


In [47]:
#数据包装
batch_size = 4
trainloader = torch.utils.data.DataLoader(data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

## Model

In [31]:
no_cuda = True
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [99]:
class EagleC_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        #卷积
        self.features_ = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
                                        ,nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                       
                                       ,nn.Conv2d(32,64,3,stride=1,padding=1)
                                        ,nn.ReLU(inplace=True)
                                       ,nn.MaxPool2d(2)
                                      )
        #分类
        #根据net输出的形状确定
        self.clf_ = nn.Sequential(nn.Dropout(0.5)
                                  ,nn.Linear(64*7*7,512)
                                  ,nn.ReLU(inplace=True)
                                 ,nn.Linear(512,6)
                                 ,nn.Sigmoid()
                                 )
    
    def forward(self,x):
        x = self.features_(x) #用特征提取的架构提取特征
        x = x.view(-1,64*7*7) #调整数据结构，拉平数据
        output = self.clf_(x)
        return output

In [100]:
model = EagleC_CNN().to(device)
#多分类
criterion = nn.CrossEntropyLoss()
#优化器
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train

In [101]:
scheduler = ReduceLROnPlateau(optimizer, 'min')

In [102]:
def train(model, device, train_loader, optimizer, clip,criterion):
	model.train()
	#损失
	loss_sum = 0.0
	for i, (data, target) in enumerate(train_loader):
		if i == (len(train_loader) - 1):
			continue
		data, target = Variable(data).to(device), Variable(target, requires_grad=False).to(device)
		#将模型的参数梯度初始化为0
		optimizer.zero_grad()
		#得到结果
		output = model(data)
		# loss function
		loss = criterion(output, target)
		loss.backward()
		#获取当前lr
		lr_current = get_lr(optimizer)
		clip2 = clip/lr_current
		#梯度裁剪
		nn.utils.clip_grad_norm_(model.parameters(),clip2)
		optimizer.step() #更新参数
		loss_sum = loss_sum + loss.item()
	#平均的损失函数
	return loss_sum/i
	
# validation
def validate(model, device, validation_loader,criterion):
	model.eval()
	loss_sum = 0.0
	with torch.no_grad():
		for i, (data, target) in enumerate(validation_loader):
			data, target = Variable(data).to(device), Variable(target, requires_grad=False).to(device)
			output = model(data)
			# loss function
			loss = criterion(output, target)
			#计算损失函数
			loss_sum = loss_sum + loss.item()
	#平均的损失函数
	return loss_sum/i

# get current learning rate
def get_lr(optimizer):
	for param_group in optimizer.param_groups:
		return param_group['lr']

In [106]:
#训练
for epoch in range(1, epochs):
    loss_train = train(model, device, train_loader, optimizer,clip,criterion) 
    loss_validate = validate(model, device, validation_loader,criterion)
    scheduler.step(loss_validate)	
    lr_current = get_lr(optimizer)
    print("epoch:{},lr:{},loss train:{},loss validate:{}".format(epoch, lr_current, np.round(loss_train,3), np.round(loss_validate,3)) ) 
    # save the model
    torch.save(model.state_dict(), "model_epoch" + str(epoch))

epoch:1,lr:0.01,loss train:1.552,loss validate:1.649
epoch:2,lr:0.01,loss train:1.551,loss validate:1.626
epoch:3,lr:0.001,loss train:1.552,loss validate:1.626
epoch:4,lr:0.001,loss train:1.551,loss validate:1.649
epoch:5,lr:0.001,loss train:1.553,loss validate:1.626
epoch:6,lr:0.001,loss train:1.552,loss validate:1.672
epoch:7,lr:0.001,loss train:1.55,loss validate:1.649
epoch:8,lr:0.001,loss train:1.551,loss validate:1.626
epoch:9,lr:0.001,loss train:1.552,loss validate:1.626


In [53]:
#save model
PATH = 'model.pth'
torch.save(model.state_dict(), PATH)

## Predict

In [ ]:
data_test = np.load()
target_test = np.load()

test_loader = torch.utils.data.DataLoader(data.TensorDataset(torch.from_numpy(data_test), torch.from_numpy(np.zeros(target_test.shape[0]))), batch_size=args.batch_size, shuffle=False)

In [ ]:
#TODO:要改
result = np.zeros((low_res_test.shape[0],1,28,28))
for i, (data, _) in enumerate(test_loader):
	data2 = Variable(data).to(device)
	output = Net(data2)
	resulti = output.cpu().data.numpy()
	resulti = np.squeeze(resulti)
	i1 = i * args.batch_size
	i2 = i1 + args.batch_size
	if i == int(low_res_test.shape[0]/args.batch_size):
		i2 = low_res_test.shape[0]
	result[i1:i2,0,:,:] = resulti

np.save(args.file_test_predicted, result)